In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando e compilando o modelo.

+ Usamos a API sequencial.

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

### Criando a callback `ModelCheckpoint`.

+ A callback `ModelCheckpoint` salva pontos de verificação do modelo em intervalos regulares durante o treinamento, por padrão no final de cada época.

+ Se usarmos um conjunto de validação durante o treinamento, podemos definir o parâmetro `save_best_only=True` ao criar a callback `ModelCheckpoint`. 

+ Nesse caso, a callback só salvará o modelo quando seu desempenho no conjunto de validação for o melhor até o momento. 

+ Dessa forma, não precisamos nos preocupar em treinar por muito tempo e sobreajustar ao conjunto de treinamento: basta restaurar o último modelo salvo após o treinamento e este será o melhor modelo no conjunto de validação. 

+ Esta é uma maneira simples de implementar a parada antecipada (*early-stop*).

In [5]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

### Treinando e avaliando o modelo salvo.

In [6]:
# Treina o modelo com a callback especificada.
history = model.fit(X_train, y_train, 
                    epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb]
                   )

# Carrega o melhor modelo no conjunto de validação.
model = keras.models.load_model("my_keras_model.h5") # rollback to best model

# Avalia o modelo.
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 [==============================] - 2s 4ms/step - loss: 1.8866 - val_loss: 0.7126
Epoch 2/10
363/363 [==============================] - 2s 5ms/step - loss: 0.6577 - val_loss: 0.6880
Epoch 3/10
363/363 [==============================] - 1s 4ms/step - loss: 0.5934 - val_loss: 0.5803
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5557 - val_loss: 0.5166
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5272 - val_loss: 0.4895
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5033 - val_loss: 0.4951
Epoch 7/10
363/363 [==============================] - 1s 4ms/step - loss: 0.4854 - val_loss: 0.4861
Epoch 8/10
363/363 [==============================] - 1s 3ms/step - loss: 0.4709 - val_loss: 0.4554
Epoch 9/10
363/363 [==============================] - 1s 4ms/step - loss: 0.4578 - val_loss: 0.4413
Epoch 10/10
162/162 [==============================] - 1s 2ms/step - loss: 0.4382


### Criando a callback `EarlyStopping`.

+ Outra maneira de implementar a parada antecipada é simplesmente usar a callback `EarlyStopping`. 

+ Ela interromperá o treinamento quando não medir nenhum progresso no conjunto de validação por várias épocas (definidas pelo parâmetro da paciência) e, opcionalmente, carrega o melhor modelo. 

+ Podemos combinar as duas callbacks para salvar os pontos de verificação do modelo (no caso de seu computador travar) e interromper o treinamento mais cedo quando não houver mais progresso (para evitar desperdício de tempo e recursos).

In [7]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

### Treinando e avaliando o modelo.

 + O número de épocas pode ser definido como um valor grande, pois o treinamento será interrompido automaticamente quando não houver mais progresso. 
 
 + Além disso, não há necessidade de restaurar o melhor modelo salvo neste caso (`restore_best_weights=True`), pois a callback `EarlyStopping` armazenará os melhores pesos e os restaurará ao final do treinamento.

In [8]:
# Treina o modelo com as callbacks definidas.
history = model.fit(X_train, y_train, 
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb]
                   )

# O melhor modelo é carregado ao final.
mse_test = model.evaluate(X_test, y_test)

Epoch 1/100
363/363 [==============================] - 2s 4ms/step - loss: 0.4393 - val_loss: 0.4110
Epoch 2/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4315 - val_loss: 0.4266
Epoch 3/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4259 - val_loss: 0.3996
Epoch 4/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4201 - val_loss: 0.3939
Epoch 5/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4154 - val_loss: 0.3889
Epoch 6/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4111 - val_loss: 0.3866
Epoch 7/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4074 - val_loss: 0.3860
Epoch 8/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4040 - val_loss: 0.3793
Epoch 9/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4008 - val_loss: 0.3746
Epoch 10/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3976 - val_lo

363/363 [==============================] - 1s 3ms/step - loss: 0.3324 - val_loss: 0.3274
Epoch 82/100
363/363 [==============================] - 2s 4ms/step - loss: 0.3320 - val_loss: 0.3167
Epoch 83/100
363/363 [==============================] - 2s 4ms/step - loss: 0.3317 - val_loss: 0.3279
Epoch 84/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3312 - val_loss: 0.3635
Epoch 85/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3310 - val_loss: 0.3174
Epoch 86/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3308 - val_loss: 0.3156
Epoch 87/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3305 - val_loss: 0.3528
Epoch 88/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3299 - val_loss: 0.3257
Epoch 89/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3294 - val_loss: 0.3632
Epoch 90/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3296 - val_loss: 

### Criando nossa própria callback.

+ Se precisarmos de controle extra, podemos criar facilmente nossas próprias callbacks. 

+ Por exemplo, a seguinte callback exibirá a proporção entre a perda de validação e a perda de treinamento durante o treinamento ao final de cada época (por exemplo, para detectarmos overfitting).

+ Notem que a classe `PrintValTrainRatioCallback` herda da classe `keras.callbacks.Callback`.

In [9]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nMinha callback -> val/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

### Treinando e avaliando o modelo.

In [10]:
val_train_ratio_cb = PrintValTrainRatioCallback()

# Treina o modelo com as callbacks definidas.
history = model.fit(X_train, y_train, 
                    epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[val_train_ratio_cb]
                   )

# O melhor modelo é carregado ao final.
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
350/363 [===========================>..] - ETA: 0s - loss: 0.3294
Minha callback -> val/train: 1.08
363/363 [==============================] - 2s 5ms/step - loss: 0.3302 - val_loss: 0.3557
Epoch 2/10
346/363 [===========================>..] - ETA: 0s - loss: 0.3314
Minha callback -> val/train: 1.00
363/363 [==============================] - 2s 5ms/step - loss: 0.3301 - val_loss: 0.3303
Epoch 3/10
353/363 [============================>.] - ETA: 0s - loss: 0.3300
Minha callback -> val/train: 1.02
363/363 [==============================] - 2s 4ms/step - loss: 0.3296 - val_loss: 0.3351
Epoch 4/10
361/363 [============================>.] - ETA: 0s - loss: 0.3293
Minha callback -> val/train: 0.95
363/363 [==============================] - 1s 4ms/step - loss: 0.3295 - val_loss: 0.3142
Epoch 5/10
355/363 [============================>.] - ETA: 0s - loss: 0.3287
Minha callback -> val/train: 1.02
363/363 [==============================] - 2s 5ms/step - loss: 0.3291 - val_loss: 0.3365


### Importante

+ Podemos implementar callbacks para cada um dos seguintes eventos:
    + on_train_begin(), 
    + on_train_end(),
    + on_epoch_begin(), 
    + on_epoch_begin(), 
    + on_batch_end(), 
    + on_batch_end().

+ Além disso, caso necessário, as callbacks também podem ser usadas durante a avaliação e a predição, por exemplo, para depuração. Para isso, devemos implementar alguns métodos.

+ Os métodos abaixo são chamados por `evaluate()`:

    + on_test_begin(),
    + on_test_end(),
    + on_test_batch_begin(), 
    + ou on_test_batch_end().

+ Os métodos abaixo são chamados por `predict()`:

    + on_predict_begin(), 
    + on_predict_end(), 
    + on_predict_batch_begin(), 
    + on_predict_batch_end().